In [1]:
%pip install transformers
%pip install PyPDF2

     |████████████████████████████████| 501kB 4.8MB/s 
     |████████████████████████████████| 1.0MB 52.4MB/s 
     |████████████████████████████████| 870kB 44.2MB/s 
     |████████████████████████████████| 3.7MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=29a076c1aa06626aea25edb0068dbac1bd8235d72d8fd913c497bd845de1958f
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 81kB 3.8MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=d92f1aa7f2a86ec260a7e55fb8251522e70b64d933539b9f3374348131fb2337
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
%rm -rf BERT_financialReportAnalysis/

In [3]:
!git clone https://github.com/francisliu2/BERT_financialReportAnalysis.git

Cloning into 'BERT_financialReportAnalysis'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 97 (delta 60), reused 62 (delta 25), pack-reused 0
Unpacking objects: 100% (97/97), done.


In [4]:
%cd BERT_financialReportAnalysis/

/content/BERT_financialReportAnalysis


In [5]:
from companytoVector_BERT import *
from corporate_profile_extrator import *

from google.colab import files
import os
import glob

import pandas as pd

import string  
punc = string.punctuation 

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopWords = list(stopwords.words('english'))
os.mkdir('outputs')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



Max number of tokens in one sentence: 512
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
CP = glob.glob("./data/*.pdf")

In [7]:
model = load_model()

In [0]:
success = []
failed = []
result_temp = [] # for debugging
for i in range(len(CP)):
  name = CP[i].replace('.pdf', '') 
  name = name.replace('./data/', '')
  corporate_profile_text = get_corporate_profile(CP[i])
  corporate_profile_tokens = text_tokening(corporate_profile_text, verbose=1)
  corporate_profile_ids = corporate_profile_tokens[0]
  if corporate_profile_ids == None:
    failed.append({name, 'sentence is too long'})
  else:
    success.append(name)
    result = company2vector(corporate_profile_ids, model, average_over_sent=False)
    result_temp.append(result)
  
    pd_temp = vector_of_each_whole_token(corporate_profile_tokens, result)
    # Remove punctuation
    pd_temp['punc'] = pd_temp.token.apply(lambda x: x in punc)

    # Remove stopwords
    pd_temp['stopwords'] = pd_temp.token.apply(lambda x: x in stopWords)

    # Save
    pd_temp.to_pickle('./outputs/'+name+'.pkl')
    # np.save('./outputs/'+name, result)

In [0]:
test = pd.read_pickle('./outputs/'+name+'.pkl')

In [0]:
test

,sentence_number,token_number,token,vector,punc,stopwords
0,0,1,sustainability,"[0.065969, 0.076606676, 0.62781805, 0.01035083...",False,False
1,0,2,report,"[0.40392673, 0.4611601, 0.9255303, -0.18665083...",False,False
2,0,3,for,"[-1.0512247, 0.20628639, 0.2880623, 0.169154, ...",False,True
3,0,4,the,"[-1.0506794, -0.5677045, -0.20221244, 0.454342...",False,True
4,0,5,financial,"[-0.5520763, -0.4105609, 0.14595497, 0.0835926...",False,False
...,...,...,...,...,...,...
355,9,19,on,"[-1.5194913, -0.03699847, 0.01903405, 0.465225...",False,True
356,9,20,22,"[-0.6731026, 0.011136717, 0.4026469, -0.203729...",False,False
357,9,21,september,"[0.00090364635, 0.09324206, -0.404369, 0.11505...",False,False
358,9,22,2015,"[-1.2094734, 0.5952442, 0.9333452, 0.3811243, ...",False,False


In [0]:
# Remove punctuation
test['punc'] = test.token.apply(lambda x: x in punc)

In [0]:
# Remove stopwords
test['stopwords'] = test.token.apply(lambda x: x in stopWords)

In [0]:
result.shape

(10, 512, 768)

In [0]:
success

['Yongnam Annual Report 2010', 'China Star_Sustainability Report']

In [0]:
!zip -r ./outputs.zip ./outputs
!sleep 2
files.download('outputs.zip')

updating: outputs/ (stored 0%)
updating: outputs/Yongnam Annual Report 2010.pkl (deflated 9%)
updating: outputs/China Star_Sustainability Report.pkl (deflated 9%)


In [0]:
files.download('outputs.zip')

In [0]:
# Check time needed to run
corporate_profile_text = get_corporate_profile(CP[0])
%time result = company2vector(corporate_profile_text, model)

CPU times: user 29.9 s, sys: 2.69 s, total: 32.6 s
Wall time: 32.7 s
